<a href="https://colab.research.google.com/github/adamFittlerGit/Risk-Identification-Thesis/blob/main/LLAVA_FineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers==4.35.2
!pip install datasets
!pip install accelerate
!pip install peft
!pip install evaluate
!pip install bitsandbytes  # For 8-bit quantization if needed
!pip install sentencepiece  # Required for some tokenizers
!pip install pillow  # For image processing
!pip install huggingface_hub  # For model uploads
!pip install tensorboard  # For monitoring training
!pip install matplotlib  # For visualizations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 95.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.35.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install transformers --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 87.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [1]:
import pandas as pd
from datasets import Dataset, load_dataset
from transformers import TrainingArguments, Trainer
from transformers import AutoProcessor, LlavaForConditionalGeneration
from PIL import Image
import pandas as pd

In [3]:
model_id = "llava-hf/llava-1.5-7b-hf"
processor = AutoProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto"
)



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
def create_dataset(csv_file, image_dir):
    data = pd.read_csv(csv_file)
    data["image"] = data["image"].apply(lambda x: image_dir + "/" + x)
    dataset = Dataset.from_pandas(data)
    return dataset

def collate_fn(batch):
    images = [Image.open(item["image"]) for item in batch]
    questions = [f"<image>\n{item['question']}" for item in batch]
    answers = [item["answer"] for item in batch]

    inputs = processor(
        text=questions,
        images=images,
        return_tensors="pt",
        padding=True,
        truncation=True
    )

    labels = processor.tokenizer(
        answers,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).input_ids

    inputs["labels"] = labels
    return inputs

training_args = TrainingArguments(
    output_dir="./llava-vqa-finetuned",
    learning_rate=5e-5,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    save_strategy="epoch",
    logging_steps=10,
    bf16=True,  # or fp16=True depending on your hardware
    gradient_checkpointing=True,
    report_to="tensorboard",
    remove_unused_columns=False,
)

# Create the datasets
train_dataset = create_dataset("/content/drive/MyDrive/uni/thesis/llava/llava/clean/csv/train.csv", "/content/drive/MyDrive/uni/thesis/llava/llava/images/folder/train")
val_dataset = create_dataset("/content/drive/MyDrive/uni/thesis/llava/llava/clean/csv/val.csv", "/content/drive/MyDrive/uni/thesis/llava/llava/images/folder/validation")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn,
)

trainer.train()

# After training is complete
trainer.save_model("./my_fine_tuned_llava")

# Also save the processor (important for handling inputs later)
processor.save_pretrained("./my_fine_tuned_llava")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


ValueError: Image features and image tokens do not match: tokens: 0, features 2304